# Introduction

This notebook shows how to run a strategy against feeds coming from the Binance crypto exchange. Roboquant includes a dedicated module for crypto trading and this module also includes the support for Binance.
There are two types of feed supported:

1) Historic feed with price bars (candlesticks)
2) Live feed with price bars

As always we load roboquant using the **%use roboquant** cell magic. This will retrieve all the required packages and imports most commonly used functionality. The Welcome() is optional, but it provides some insights into the used environment and available memory. 

In [ ]:
%use roboquant
Welcome()

# Setup 
By default the required Binance classes are not imported, as is the case with all third party integrations, so lets do that first.

In [ ]:
import org.roboquant.binance.*

We now create the components we want to use. We start with defining a simple strategy that uses moving average. Because we won't be using USD as a base currency, we create a new SimBroker instance with an initial deposit of 1_000.000,- Binance USD (BUSD). We could also have used the default SimBroker with 1 million USD initial deposit. But in that case we would have needed to define the exchange rate from USD to BUSD, since we going to trade Bitcoin denominated in BUSD. 

In [ ]:
val strategy = EMACrossover(3, 5)
val broker = SimBroker.withDeposit(1_000_000.00, "BUSD")
val roboquant =  Roboquant(strategy, ProgressMetric(), broker = broker)

# Binance Historic Feed
We start with retrieving historic data from Binance. Please note how to define a currency-pair: currency1-currency2

In [ ]:
val feed = BinanceHistoricFeed()
val tf = TimeFrame.pastPeriod(800)
feed.retrieve("BTC-BUSD", timeFrame = tf)
feed.timeFrame

Lets plot the data we just received. Since the feed can contain multiple assets, we also need to tell the chart which asset to draw (although in this case there is only one).

In [ ]:
PriceBarChart(feed, feed.assets.first())

In [ ]:
roboquant.run(feed)

In [ ]:
roboquant.logger.summary()

In [ ]:
roboquant.broker.account.summary()

We will now plot again the historic prices but this time also plot the actual trades that happened as part of our run. So you can easily see when a trade occured in time and the amount traded.

In [ ]:
PriceChart(feed, feed.assets.first(), broker.account.trades)

# Binance Live Feed
Now we create an instance of the BinanceFeed and subscribe to pricebar events (candlesticks) of the the BTC/BUSD currency pair. It is also possible to subscribe to multiple currency pairs by adding additional parameters to the same *subscribePriceBar* method.

In [ ]:
val feed = BinanceFeed()
feed.subscribePriceBar("BTC-BUSD")

Because we won't be using USD as a base currency, we create a new Simbroker instance with an initial deposit of 1,000,000.- Binance USD (BUSD). We also define metrics that will help afterwards to validate the performance:

1) ProgressMetric: overall progress of the run
2) AccountSummary: state of the account during the run
2) PriceMetric: capture the recorded prices for an asset

In [ ]:
val strategy = EMACrossover(3, 5)
val broker = SimBroker.withDeposit(1_000_000.00, "BUSD")
val priceMetric = PriceMetric(feed.assets.first())
val roboquant =  Roboquant(strategy, ProgressMetric(), AccountSummary(), priceMetric, broker = broker)

All that remains is to start the run. Because it is a live feed we have to specify for how long we want to run it otherwise it would run forever or you interrupt the execution. In this case we run it for 30 minutes, but you can run it of course for any duration. If you run it for a short time, it is less likely that the used strategy will generate signals and as a result you won't see any trades or changes to the portfolio.

In [ ]:
val timeFrame = TimeFrame.nextMinutes(30)
roboquant.run(feed, timeFrame)

In [ ]:
roboquant.logger.summary()

In [ ]:
PriceBarChart(priceMetric, priceMetric.asset)

In [ ]:
PriceChart(priceMetric, priceMetric.asset, roboquant.broker.account.trades)

In [ ]:
val accountValue = roboquant.logger.getMetric("account.value")
MetricChart(accountValue)

# Binance Trading
This is still work in progress and is currently not available. However the goal is that in the end you can also place orders on Binance using roboquant. There is already some ground work done, so have a look at the roboquant source code if you want to see what is already there and perhaps contribute some code. 